### ISS VSE CA2 Part2

#### By: Kenneth Goh (A0198544N), Raymond Ng (A0198543R), Tan Heng Han (A0198502B)

In [78]:
import cv2
import os
import math
import time
from collections import Counter
from itertools import repeat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import Adam

import sklearn.metrics as metrics

### Load and slice videos for training
Slice training videos into 5hz frames for model input
Data set:
1. Push up videos
2. Mixture of non push up videos

In [77]:
# Get filenames and labels for data
labels = ['PushUps', 'NotPushUps']
data_path = 'data/'
train = pd.DataFrame()
filenames = []
for lbl in labels:
    filename = np.array(glob(os.path.join(data_path, lbl) + '/*.avi'))
    filenames.append(filename)

train['video_name'] = [item for sublist in filenames for item in sublist]
train

,video_name
0,data/PushUps\v_PushUps_g01_c01.avi
1,data/PushUps\v_PushUps_g01_c02.avi
2,data/PushUps\v_PushUps_g01_c03.avi
3,data/PushUps\v_PushUps_g01_c04.avi
4,data/PushUps\v_PushUps_g01_c05.avi
...,...
197,data/NotPushUps\v_PullUps_g24_c04.avi
198,data/NotPushUps\v_PullUps_g25_c01.avi
199,data/NotPushUps\v_PullUps_g25_c02.avi
200,data/NotPushUps\v_PullUps_g25_c03.avi


In [75]:
# Slice and store the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 2 == 0):
            # storing the frames in a new folder named train_1
            filename ='data/images/' + videoFile.split('\\')[1] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [00:36<00:00,  5.47it/s]


### Load OpenPose models

#### Instructions to get openpose models
1. Clone https://github.com/CMU-Perceptual-Computing-Lab/openpose
2. Open a console and run **openpose/models/getModels.bat**
3. Copy contents from **openpose/models/pose/mpi to Model/mpi** and **openpose/models/pose/coco to Model/coco**

In [25]:
# Model Selection
OPENPOSE_MODEL = 'mpi'

In [26]:
if OPENPOSE_MODEL == 'coco':
    PROTO_FILE = 'Model/pose_deploy_linevec.prototxt'
    WEIGHTS_FILE = 'Model/pose_iter_440000.caffemodel'
elif OPENPOSE_MODEL == 'mpi':
    PROTO_FILE = 'Model/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
    WEIGHTS_FILE = 'Model/mpi/pose_iter_160000.caffemodel'
else:
    raise Exception('Open pose model not recognised, choose either mpi or coco')

In [27]:
if OPENPOSE_MODEL == 'coco':
    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                   "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                   "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
                   "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }
    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                   ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                   ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                   ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]
elif OPENPOSE_MODEL == 'mpi':
    BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                  "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                  "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                  "Background": 15}
    POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                   ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                   ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]
else:
    raise Exception('Open pose model not recognised, choose either mpi or coco')

In [28]:
# Load openpose model via Opencv
try:
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
except:
    raise Exception('Error reading model, check model proto and weights file.')

In [4]:
dataset_frame_path = 'train_1'
frame_filenames = glob(dataset_frame_path + '\*.jpg')

trainFrame = pd.DataFrame()
trainFrame['frames'] = frame_filenames
trainFrame.shape

(28, 1)

In [5]:
inWidth = 368
inHeight = 368

imgPaths = []
for frame in trainFrame['frames']:
    img = cv2.imread(frame)
#     img = cv2.resize(img, (inWidth, inHeight))
    imgPaths.append(img)
    
# cv2.imshow('img', imgPaths[0])
# cv2.waitKey(3)

testimg = imgPaths[0]
frameWidth = testimg.shape[1]
frameHeight = testimg.shape[0]

In [6]:
# inBlob = cv2.dnn.blobFromImage(imgPaths[1], 1.0/255, (imgPaths[1].shape[1], imgPaths[1].shape[0]), (0,0,0), swapRB=False, crop=False)
inBlob = cv2.dnn.blobFromImage(testimg, 1.0/255, (368, 368), (0,0,0), swapRB=False, crop=False)
net.setInput(inBlob)
output = net.forward()

In [7]:
kwinName="Pose"
cv2.namedWindow(kwinName, cv2.WINDOW_AUTOSIZE)

In [10]:
points = []
for i in range(len(BODY_PARTS)):
    # Slice heatmap of corresponging body's part.
    heatMap = output[0, i, :, :]
 
    # Originally, we try to find all the local maximums. To simplify a sample
    # we just find a global one. However only a single pose at the same time
    # could be detected this way.
    _, conf, _, point = cv2.minMaxLoc(heatMap)
    x = (frameWidth * point[0]) / output.shape[3]
    y = (frameHeight * point[1]) / output.shape[2]
 
    # Add a point if it's confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > 0.1 else None)

In [11]:
for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)
 
    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]
    if points[idFrom] and points[idTo]:
        cv2.line(testimg, points[idFrom], points[idTo], (255, 74, 0), 3)
        cv2.ellipse(testimg, points[idFrom], (4, 4), 0, 0, 360, (255, 255, 255), cv2.FILLED)
        cv2.ellipse(testimg, points[idTo], (4, 4), 0, 0, 360, (255, 255, 255), cv2.FILLED)
        cv2.putText(testimg, str(idFrom), points[idFrom], cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv2.LINE_AA)
        cv2.putText(testimg, str(idTo), points[idTo], cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv2.LINE_AA)

In [13]:
cv2.imshow(kwinName, testimg)
cv2.waitKey(0)
cv2.destroyAllWindows()